# TensorFlow and other tools for ML in Julia

**Lyndon White**
 - Research Software Engineer -- Invenia Labs, Cambridge
 - Technically still PhD Candidate -- The University of Western Australia


In [1]:
using Pkg: @pkg_str
pkg"activate  ."

In [2]:
using TensorFlow
using TensorFlow: summary

┌ Info: Recompiling stale cache file /Users/oxinabox/.julia/compiled/v1.1/TensorFlow/IhIhf.ji for TensorFlow [1d978283-2c37-5f34-9a8e-e9c0ece82495]
└ @ Base loading.jl:1184
┌ Warning: Loading a new version of TensorFlow.jl for the first time. This initial load can take around 5 minutes as code is precompiled; subsequent usage will only take a few seconds.
└ @ TensorFlow ~/Documents/talks/JuliaDeepLearningMeetupLondon2019/dev/TensorFlow/src/TensorFlow.jl:3
┌ Warning: Module Compat with build ID 259425757446860 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:947
┌ Warning: Module Compat with build ID 259425757446860 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:947
┌ Warning: Loading a new version of TensorFlow.jl for the firs

## 4 Types of Nodes, i.e. `Tensors`
 - **Placeholders:** this is where you put your inputs
 - **Operations:** theres transform inputs into outputs, they do math
 - **Variables:** thes arre the things you train, they are mutable
 - **Actions:** These are operations with side effects, like logging (TensorBoard)) and mutating Variable (Optimizers)

## Functions

Functions mutate **the graph** to introduce nodes.

For example:
 - `sin(::Float64)` in julia would return a `Float64` that is the answer.
 - `sin(::Tensor)` introduces a `sin` operation into the graph, and returns a `Tensor` that is a reference to it's output, this could be feed to other operations.
 
The answer to that operation is not computed, until you execute the graph.


In [3]:
sess= Session(Graph())

@tf begin
    x = placeholder(Float64)
    y = sin(x)
end

@show y

run(sess, y, Dict(x=>0.5))

y = <Tensor y:1 shape=unknown dtype=Float64>


2019-01-29 22:23:46.479161: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.2 AVX AVX2 FMA


0.479425538604203

## Automatic Node Naming

 - Notice before I did `@tf begin ... end`
 - **This is not at all required**
 - But it does enable automatic node naming
 - so `@tf y = sin(x)` actually becomes `y = sin(x; name="y")`
 - This gives you a good graph in tensorboard, and also better error messages.
 - Further it lets us look up tensors from the graph by **name**

In [4]:
@show sess.graph["x"]
@show sess.graph["y"]

run(sess, sess.graph["y"], Dict(sess.graph["x"]=>0.5))

sess.graph["x"] = <Tensor x:1 shape=unknown dtype=Float64>
sess.graph["y"] = <Tensor y:1 shape=unknown dtype=Float64>


0.479425538604203

In [5]:
using TensorFlow
using MLDataUtils
using MLDatasets

using ProgressMeter

In [6]:
leaky_relu6(x) = 0.01x + nn.relu6(x)

leaky_relu6 (generic function with 1 method)

In [7]:
sess = Session(Graph())

# Network Definition
@tf begin
    X = placeholder(Float32, shape=[-1, 28*28])
    
    # Network parameters
    
    hl_sizes = [512, 128, 64, 2, 64, 128, 512]

    Zs = [X]
    for (ii, hlsize) in enumerate(hl_sizes)
        Wii = get_variable("W_$ii", [get_shape(Zs[end], 2), hlsize], Float32)
        bii = get_variable("b_$ii", [hlsize], Float32)
        Zii = leaky_relu6(Zs[end]*Wii + bii)
        push!(Zs, Zii)
    end
    
    Wout = get_variable([get_shape(Zs[end], 2), 28*28], Float32)
    bout = get_variable([28*28], Float32)
    Y = nn.sigmoid(Zs[end]*Wout + bout)
    
    
    
    Z_code = Zs[end÷2] # A name for the coding layer
    has_died = reduce_any(reduce_all(Z_code.==0f0, axis=2))
end


<Tensor has_died:1 shape=() dtype=Bool>

In [9]:
losses = 0.5(Y .- X).^2
loss = reduce_mean(losses)

<Tensor reduce_2:1 shape=() dtype=Float64>

In [ ]:
#optimizer = train.minimize(train.AdamOptimizer(), loss)
optimizer = train.minimize(train.GradientDescentOptimizer(0.00001), loss)


In [ ]:

run(sess, global_variables_initializer())
auto_loss = Float64[]
@showprogress for epoch in 1:75
    epoch_loss = Float64[]
    for batch_x in eachbatch(train_images, 1_000, ObsDim.Last())
        loss_o, _ = run(sess, (loss, optimizer), Dict(X=>batch_x'))
        push!(epoch_loss, loss_o)
    end
    push!(auto_loss, mean(epoch_loss))
    #println("Epoch $epoch loss: $(auto_loss[end])")
    
    ### Check to see if it died
    if run(sess, has_died, Dict(X=>train_images'))
        error("Neuron in hidden layer has died, must reinitialize.")
    end
end

In [12]:
# Create a summary writer
summary_writer = TensorFlow.summary.FileWriter(mkpath("logs"))
run(session, TensorFlow.summary.merge_all())
write(summary_writer, , 1)

TensorFlow.TFException: Tensorflow error: Status: Value for attr 'N' of 0 must be at least minimum 1
	; NodeDef: {{node MergeSummary}} = MergeSummary[N=0](); Op<name=MergeSummary; signature=inputs:N*string -> summary:string; attr=N:int,min=1>


In [5]:
pkg"add MLDataUtils"

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating registry at `~/.julia/registries/Invenia`
  Updating git-repo `git@gitlab.invenia.ca:invenia/PackageRegistry.git`
 Resolving package versions...
 Installed Conda ── v1.2.0
 Installed Compat ─ v1.5.0
  Updating `~/Documents/talks/JuliaDeepLearningMeetupLondon2019/Project.toml`
  [cc2ba9b6] + MLDataUtils v0.4.0
  [1d978283] ? TensorFlow v0.11.0+ [`dev/TensorFlow`] ⇒ v0.12.0 [`dev/TensorFlow`]
  Updating `~/Documents/talks/JuliaDeepLearningMeetupLondon2019/Manifest.toml`
  [324d7699] + CategoricalArrays v0.5.2
  [34da2185] ↑ Compat v1.4.0 ⇒ v1.5.0
  [8f4d0f93] ↑ Conda v1.1.1 ⇒ v1.2.0
  [a93c6f00] + DataFrames v0.17.0
  [9a8bc11e] + DataStreams v0.4.1
  [82899510] + IteratorInterfaceExtensions v0.1.1
  [7f8f8fb0] + LearnBase v0.2.2
  [9920b226] + MLDataPattern v0.5.0
  [cc2ba9b6] + MLDataUtils v0.4.0
  [66a33bbf] + MLLabelUtils v0.5.1
  [dbb5928d] + MappedArr

# Invenia Labs
## We're hiring
### People who know Julia
### People who know Machine Learning
I have left some fliers about open positions at the entrance.